In [1]:
from hiresprv.auth import login
from hiresprv.archive import Archive
from hiresprv.database import Database
import pandas as pd
from hiresprv.idldriver import Idldriver
from hiresprv.download import Download
import numpy as np

In [2]:
login('prv.cookies')

koa = Archive('prv.cookies')

idl = Idldriver('prv.cookies')

KOA userid: jgussman
KOA Password: ········
Successful login as jgussman


In [3]:
state = Database('prv.cookies')

In [4]:
starsNamedf = pd.read_csv("spocData/starnames_crossmatch_SPOCS_NEXSCI.txt",sep=" ",header=None)
#Need to figure out all the different Catalogs we have here so I can remove them when it is time to reduce
differentIndentifiers = ["HD","CK","HTR","KEPLER","HIP","_","KIC"
                        ,"MARVELS","WASP","XO","BD+","TRES","EPIC","K"
                        ,"HII","BD-","HAT","COROT"]
HIRESstarNamesList = []
for starNames in starsNamedf.to_numpy():
    HIRESstarNamesList.append(starNames[1])
    idFound = 0 
    for identifier in differentIndentifiers:
        if identifier in starNames[1] and idFound == 0: 
            idFound = 1
    if idFound == 0:
        print(starNames[1])

In [5]:
successStars = []
failureStars = []

In [10]:
bigq= ""
for TargetName in HIRESstarNamesList:
    star_info = state.target_info(TargetName)

    df = pd.read_html(star_info, header=0)[0]
    #Drop all the rows that are NOT RV observations 
    df = df.drop(df[(df['OBTYPE'] != 'RV\xa0observation')].index)
    if not df.empty: #If the star has no RV observations. It probably just has a template
        try:
            filename = df['FILENAME'][0]
            filename = filename.replace("r","")
            filename = filename.split(".")[0]
        except: 
            filename = df['FILENAME'].array[0]
            filename = filename.replace("r","")
            filename = filename.split(".")[0]
        #Create the Querry to Extract the RV Data 
        querry_To_NEXsci = "template {0} {1}".format(TargetName,filename)

        for RVObv in df.to_numpy():
            querry_To_NEXsci += '\nrv {0} {1}'.format(TargetName,RVObv[3])

        querry_To_NEXsci += "\nrvcurve {0}".format(TargetName)

        
        bigq += querry_To_NEXsci



URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

In [15]:
rtn = idl.run_script(bigq) 

status= ok
msg= Script running in background. Consult monitor for status.


In [26]:
#Download the Data
data = Download('prv.cookies', './')
somethingWentWrongStarList = []
for s in HIRESstarNamesList:
    rtn = data.rvcurve(s)
    try:
        with open('vst{0}.csv'.format(s), 'r') as file:
          for line in file:
            print(line, end='')
    except:
        somethingWentWrongStarList+=[s]
        
        

{"status":"Failed to open requested file vst11964A.csv in your workspace."}
FILENAME,BJD_TDB,RV,RV_ERR,BC,ADU,CHI2
"r20040821.89",13238.78941400023,19.08408359153110,1.04921,-5384.812011718750,118152,1.48378
"r20040821.92",13238.79531699978,21.27154753673770,1.10759,-5399.720214843750,118655,1.47474
"r20040823.176",13240.92182599986,24.82566416655510,1.08642,-6376.454101562500,122936,1.52236
"r20041023.177",13301.87702499982,18.02334630373982,1.05958,-19662.63867187500,102103,1.16299
"r20041023.178",13301.87814699998,18.68271215233347,1.04652,-19663.57617187500,101678,1.15475
"r20041024.126",13302.71053599985,18.76719378118480,0.924400,-19440.05273437500,102807,1.14278
"r20041024.127",13302.71184400003,16.31592774963752,0.877087,-19443.19531250000,102977,1.17621
"r20041025.123",13303.69765899982,13.99977080498739,1.18745,-19479.40625000000,39270,1.07233
"r20041025.124",13303.70569099998,18.22003941136661,0.909028,-19498.82226562500,102514,1.11892
"r20041025.125",13303.70793699985,16.53

{"status":"Failed to open requested file vst33334.csv in your workspace."}
{"status":"Failed to open requested file vst34887.csv in your workspace."}
{"status":"Failed to open requested file vst36130.csv in your workspace."}
{"status":"Failed to open requested file vst51046.csv in your workspace."}
{"status":"Failed to open requested file vst64324.csv in your workspace."}
{"status":"Failed to open requested file vst70516.csv in your workspace."}
{"status":"Failed to open requested file vst41700.csv in your workspace."}
{"status":"Failed to open requested file vst69960.csv in your workspace."}
{"status":"Failed to open requested file vst73534.csv in your workspace."}
{"status":"Failed to open requested file vst33822.csv in your workspace."}
{"status":"Failed to open requested file vst38467.csv in your workspace."}
{"status":"Failed to open requested file vst37986.csv in your workspace."}
FILENAME,BJD_TDB,RV,RV_ERR,BC,ADU,CHI2
"r20041229.277",13369.04973400012,-30.51550299383487,0.997272

{"status":"Failed to open requested file vst109409.csv in your workspace."}
FILENAME,BJD_TDB,RV,RV_ERR,BC,ADU,CHI2
"r20050419.118",13479.97562899999,-16.14220792594334,1.45041,-12515.56542968750,89760,1.16703
"r20051220.322",13725.15530700004,-15.53879327470745,2.41599,28963.94140625000,19392,1.04984
"r20060210.259",13777.05815599998,1.833749704874796,2.02156,19214.34570312500,29645,1.09311
"r20060312.139",13806.94328900008,-14.39113507066560,1.91683,6083.407226562500,94362,1.25281
"r20060412.189",13837.96069099987,-1.827389739248773,1.92242,-9175.947265625000,28910,1.10230
"r20060416.146",13841.84014600003,-6.684623911985398,2.09225,-10635.58593750000,23182,1.09085
"r20060710.123",13926.79242100008,4.341306780169901,2.06603,-27271.05664062500,26846,1.05539
"r20060711.125",13927.77693999978,-10.12644408054717,1.94875,-27123.67773437500,26603,1.07982
"r20060715.132",13931.81490400014,-12.77882864657511,1.94565,-26623.18359375000,26558,1.04490
"r20060716.136",13932.76668399991,-18.532390

{"status":"Failed to open requested file vst17354.csv in your workspace."}
{"status":"Failed to open requested file vst20155.csv in your workspace."}
{"status":"Failed to open requested file vst18702.csv in your workspace."}
FILENAME,BJD_TDB,RV,RV_ERR,BC,ADU,CHI2
"r20041130.181",13339.96753800008,8.822473597735737,1.57944,-4449.603027343750,94739,1.18734
"r20050127.129",13397.76105500013,-4.373132216041888,1.09715,-26017.67578125000,91630,1.14790
"r20050128.211",13398.89081799984,-2.809116002296045,1.00673,-26549.87695312500,90732,1.12774
"r20050130.117",13400.72711399989,-9.344271308925110,1.10132,-26485.08593750000,90934,1.11046
"r20101115.195",15516.09806399979,7.578533750769807,1.49802,2875.295898437500,70758,1.21172
FILENAME,BJD_TDB,RV,RV_ERR,BC,ADU,CHI2
"r20041130.179",13339.96192899998,-43.07353378367822,1.84231,-4222.268066406250,94901,1.20373
"r20050227.108",13428.79824599996,-57.86480596241532,2.08389,-29953.72851562500,41321,1.07729
"r20060814.231",13962.12523799995,-5.87158

FILENAME,BJD_TDB,RV,RV_ERR,BC,ADU,CHI2
"r20041024.191",13302.95687700016,-1.394293339831969,1.15660,12518.67578125000,113092,1.41397
"r20041024.192",13302.95753700007,-4.562375068860537,1.09299,12516.75488281250,104497,1.39364
"r20041130.182",13339.96957299998,-5.887008209572446,1.09885,-4219.264160156250,113295,1.31656
"r20041130.183",13339.97037100000,-5.345135636076311,1.19784,-4221.709960937500,111353,1.32404
"r20041130.184",13339.97104300000,-7.036259663951297,1.26305,-4223.763183593750,108552,1.29823
"r20041229.257",13368.97100300016,-7.333247586345863,1.15033,-16672.25781250000,104107,1.29549
"r20041229.258",13368.97162800003,-4.609441732566779,1.16389,-16673.65039062500,113500,1.31716
"r20041229.259",13368.97225299990,-4.938666637124996,1.19420,-16675.03710937500,107334,1.32776
"r20050225.139",13426.80399799999,-5.616403453930445,1.03055,-26619.34960937500,105919,1.35428
"r20050225.140",13426.80463400017,-2.801197080060490,1.01772,-26620.53320312500,101911,1.29864
"r20050225.14

...

FILENAME,BJD_TDB,RV,RV_ERR,BC,ADU,CHI2
"r20050127.167",13397.93101100018,78.56564479661742,1.61050,-2241.444091796875,91033,1.23264
"r20060209.168",13775.85957799992,-35.31469473800004,1.62806,-5408.821777343750,93570,1.22681
"r20061015.271",14024.13404700020,11.07000734839799,2.04314,15016.78710937500,26640,1.08986
"r20061215.198",14084.97316499986,13.61779251095188,2.02622,8896.243164062500,26226,1.06181
"r20070129.157",14129.88193800021,30.94881347988058,1.69932,-2529.492919921875,62667,1.14766
"r20071024.226",14398.12905699993,52.42234507718331,1.30836,15073.05859375000,69417,1.14321
"r20090107.182",14839.02168500004,57.30894659965797,1.81241,2949.799072265625,44126,1.07886
"r20091128.113",15164.09193499992,76.79093180175570,1.50920,11875.41308593750,44618,1.09817
"r20101121.194",15522.04438399989,-58.71778204498781,1.54253,13086.89648437500,45987,1.12898
"r20101212.163",15543.03405799996,-51.06332337574497,1.49418,9390.669921875000,45445,1.12156
"r20101214.161",15545.12415800011,-

{"status":"Failed to open requested file vstXO-1.csv in your workspace."}
FILENAME,BJD_TDB,RV,RV_ERR,BC,ADU,CHI2
"r20061214.232",14084.08366299979,-87.37394908866821,3.67466,20373.70507812500,66784,1.18557
"r20081108.179",14779.05696999980,-18.86333873490845,3.57087,28582.94335937500,26079,1.08593
"r20090602.73",14984.75763300015,71.29545582949527,5.66953,-23755.03125000000,8614,1.02800
"r20100202.144",15229.89071599999,41.00241970348662,5.29175,-2533.217041015625,9130,1.01839
"r20131020.175",16586.12277999986,-4.006928613828208,3.85685,28401.93164062500,22658,1.09798
"r20131022.226",16588.12668399978,-0.2546789775307161,3.62745,28543.14257812500,22564,1.10409
"r20131117.198",16614.07530699996,-22.77058479311735,3.72000,27473.70507812500,22460,1.09913
"r20140221.97",16709.85123000015,-10.32236855278689,4.98051,-11703.17089843750,9182,1.03601
"r20140221.113",16709.90044099977,15.42875706238397,4.98844,-11860.51171875000,9194,1.04951
FILENAME,BJD_TDB,RV,RV_ERR,BC,ADU,CHI2
"r20081205.162"

FILENAME,BJD_TDB,RV,RV_ERR,BC,ADU,CHI2
"r20041129.247",13339.10934800003,1.736374999444052,0.953578,14369.09179687500,103205,1.33564
"r20060115.182",13750.85395599995,-28.64878474022694,0.871464,-7868.221191406250,101559,1.37863
"r20060116.180",13751.91651900019,15.89152268422679,1.29816,-8557.192382812500,22438,1.10473
"r20071230.143",14464.95006200019,-11.85731131982847,1.03054,167.2890014648438,72884,1.29356
"r20081210.139",14810.93180100014,34.68547876843427,1.10737,9848.295898437500,46503,1.14881
"r20100102.167",15198.99191700015,-26.76816569735909,1.17235,-1764.863037109375,47976,1.15234
"r20100830.158",15439.13445599983,29.42616494214781,1.01831,24409.56054687500,47793,1.21325
"r20110221.135",15613.95138699980,-5.431349901753519,0.979178,-23244.40429687500,47873,1.17432
"r20120401.99",16018.79121800000,-23.13237952211406,1.02538,-28653.55859375000,48474,1.11201
"r20121007.137",16208.02275299979,24.77363541754238,1.09492,28524.84765625000,49187,1.25850
"r20131023.168",16589.09649

FILENAME,BJD_TDB,RV,RV_ERR,BC,ADU,CHI2
"r20050224.181",13425.95619700011,-8.059187528464918,0.961401,19768.17187500000,94032,1.20478
"r20050225.192",13426.99507299997,21.20981666397540,0.930880,19397.77343750000,93984,1.21277
"r20050226.247",13428.10711099999,19.12808698902592,1.01082,18850.18164062500,49568,1.13923
"r20050227.196",13429.05853699986,-9.647833723047597,1.06952,18664.02343750000,45405,1.12215
"r20050227.197",13429.06486799987,-9.126556345713595,0.977103,18649.33203125000,54054,1.12215
"r20050228.93",13429.96660300018,-28.17190826556524,0.935419,18580.18554687500,80948,1.18655
"r20050301.113",13431.09314099979,-10.49016416640828,0.875103,17980.50195312500,93285,1.21739
"r20050418.133",13478.93722200021,12.37812478746933,1.00859,-534.9739990234375,90105,1.21658
"r20050420.161",13480.84784199996,10.92761813850050,0.891716,-1168.538940429688,88574,1.28301
"r20050423.146",13483.83513800008,-2.943570877013997,0.978014,-2412.383056640625,88922,1.21123
"r20050625.125",13546.7491

FILENAME,BJD_TDB,RV,RV_ERR,BC,ADU,CHI2
"r20100718.114",15396.09910500003,-12.93202207076071,2.53117,27300.27734375000,46113,1.06366
"r20100719.100",15397.12824899983,-29.85120788552130,2.48594,27141.25390625000,44991,1.11026
"r20100722.85",15400.06830599997,-2.833019932819923,2.50980,26998.23242187500,45700,1.07393
"r20100723.56",15401.00300800009,42.71792527853292,2.47932,27034.19726562500,45851,1.06218
"r20100728.129",15406.08011100022,-13.31823136331556,2.38266,26187.12890625000,45433,1.10545
"r20100802.122",15411.07482200023,-20.99692957900331,2.61893,25354.65429687500,45254,1.12465
"r20100803.90",15412.00022399984,-21.91093805072367,2.29086,25353.21484375000,45328,1.11447
"r20100804.88",15413.04244100023,-25.07960034774254,2.28992,25051.17773437500,45262,1.09454
"r20100805.94",15414.02336699981,28.31416686130082,2.12856,24894.87890625000,45149,1.07918
"r20100817.130",15426.12031600019,16.45841311841970,2.54004,21586.43164062500,44738,1.13448
"r20100818.157",15427.11324599991,15.32

1194